In [ ]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

f_wk = "114"

In [ ]:
# Slice Dimension Attributes defined in the plugin. Please check all queries and replace <KEY HERE> with a valid name.
# For example: If slice is defined by Version.[Version Name] and Time.[Month]
# input_df = ibpl Select ([Version].[Version Name].[<KEY HERE>] * [Time].[Month].[<KEY HERE>] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;
#                             update <KEY HERE> to valid names
# input_df = ibpl Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Month].[January] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;

_tran_df = "Select ([Version].[Version Name] * [Sales Domain].[Sales Org] * [Sales Domain].[Customer Group] * [Location].[Location] *  [EPM].[EPM] * [Time].[Week] * [Item].[L6].[<KEY HERE>] * [Item].[Planning Item] ) on row, ({Measure.[Actual Shifted]}) on column;"
_prod_df = "Select ([Item].[Item] * [Item].[L3] * [Item].[L4] * [Item].[L5] * [Item].[L6].[<KEY HERE>]);"
_loc_df = "Select ([Location].[Location] * [Location].[Location Type] * [Location].[Location Region]);"
_time_df = "Select ([Time].[Day] * [Time].[Week]);"
_week_df = "Select (&CurrentWeek.element(0)*[Version].[Version Name]);"
_calendar_df = "Select ([Version].[Version Name] * [Time].[Week] * [Sales Domain].[Country] ) on row, ({Measure.[Holiday Type], Measure.[Is Holiday]}) on column;"
_segment_df = "Select ([Version].[Version Name] * [EPM].[Category] * [Sales Domain].[Sales Org] * [Item].[Planning Item] ) on row,  ({Measure.[Product Segment]}) on column;"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
O9DataLake.register("tran_df",DataSource.LS, ResourceType.IBPL, _tran_df)
O9DataLake.register("prod_df",DataSource.LS, ResourceType.IBPL, _prod_df)
O9DataLake.register("loc_df",DataSource.LS, ResourceType.IBPL, _loc_df)
O9DataLake.register("time_df",DataSource.LS, ResourceType.IBPL, _time_df)
O9DataLake.register("week_df",DataSource.LS, ResourceType.IBPL, _week_df)
O9DataLake.register("calendar_df",DataSource.LS, ResourceType.IBPL, _calendar_df)
O9DataLake.register("segment_df",DataSource.LS, ResourceType.IBPL, _segment_df)

In [ ]:
################################################################################
#####--IMPORTS--#####

import os
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta, date

import gc

import sklearn
from o9_common_utils.O9DataLake import O9DataLake
from xgboost import XGBRFRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time
import category_encoders as ce

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

################################################################################
#####--PARAMETERS--#####

cwd = os.getcwd().split(":")[0]

if cwd in ["C", "D", "E", "F"]:
    env = "local"

else:
    env = "tenant"
    logger = logging.getLogger("o9_logger")
    logger.info("Environment:")
    logger.info(env)

pd.set_option('display.max_columns', 50)

#####Define the starting lag2 week and backtest cycles####
# current_week = '2021-W43'
# cycles = 5

"""backtest here for 8 cycles. So cycle would end for week 2021-W34 which means 
we will be in W31 and have data till W30"""

###Set number of weeks of forecast (forecast length)
try:
    f_wk = int(f_wk)
except:
    logger.debug(
        "Integer value not entered for f_wk, taking default value of 114")
    f_wk = 114

if f_wk > 114:
    f_wk = 114

################################################################################
#####--DATA IMPORT--#####

###passing proper datatypes for import###
tran_df_dtypes = {
    "Qty": "float",
    "Location": "str",
    "Item": "str",
    "Customer": "str",
    "Sales Org": "str"
}

prod_df_dtypes = {
    "Item": "str",
    "L6": "str"
}

loc_df_dtypes = {
    "Location": "str"
}

if env == "local":

    reg = 'US'

    # cwd = 'D:/o9/Kraft Heinz/ML/HL MLL Code/'

    cwd = 'C:/Users/gaurang.makhaira/OneDrive - o9 Solutions/Documents/Kraft Heinz/'

    os.chdir(cwd)

    ###getting actual shifted values in tran_df###
    tran_df = pd.read_csv(cwd + 'shipment1.csv', dtype=str)
    tran_df.columns = ['Version', 'Sales Org', 'Customer', 'Location', 'EPM',
                       'WeekStr', 'L6', 'Item', 'Qty']

    version = tran_df['Version'].unique()[0]
    country = tran_df['Sales Org'].unique()[0][:2]
    salesorg = tran_df['Sales Org'].unique()[0]
    cat = tran_df['L6'].unique()[0]
    del tran_df['Version']
    tran_df = tran_df.astype(tran_df_dtypes)

    ###getting product master data###
    prod_df = pd.read_csv(cwd + 'prod1.csv', dtype=str)
    prod_df = prod_df[
        ['Item.[Item]', 'Item.[L3]', 'Item.[L4]', 'Item.[L5]', 'Item.[L6]']]
    prod_df.columns = ['Item', 'L3', 'L4', 'L5', 'L6']
    # prod_df = prod_df.astype(prod_df_dtypes)

    ###getting location master data###
    loc_df = pd.read_csv(cwd + 'loc1.csv', dtype=str)
    loc_df = loc_df[['Location.[Location]', 'Location.[Location Type]',
                     'Location.[Location Region]']]
    loc_df.columns = ['Location', 'LocType', 'Region']
    # loc_df = loc_df.astype(loc_df_dtypes)

    ###getting time master data###
    time_df = pd.read_csv(cwd + 'time1.csv')
    # time_df = pd.read_csv(cwd + 'Time.csv')

    temp = time_df.groupby('Time.[Week]').first()
    time_df = pd.merge(temp['Time.[Day]'], time_df, on='Time.[Day]', how='left')

    time_df = time_df[['Time.[Week]', 'Time.[Day]']]
    time_df['Time.[Day]'] = pd.to_datetime(time_df['Time.[Day]'],
                                           format='%d-%b-%Y')
    # time_df['Time.[Day]'] = pd.to_datetime(b['Time.[Day]'], format='%Y-%b-%d')

    # time_df = time_df[['Time.[Week]', 'Time.[WeekKey]']]
    time_df.columns = ['WeekStr', 'Week']
    time_df['Week'] = pd.to_datetime(time_df['Week'])
    time_df = time_df.drop_duplicates()

    ###getting current week###
    week_df = pd.read_csv(cwd + 'weekflag1.csv')
    weeklist_bt_df_1 = week_df.copy()

    weeklist_bt_df_1['Time.[Week]'] = weeklist_bt_df_1[
        'Time.[Week]'].str.replace('-W', '')

    current_week = weeklist_bt_df_1['Time.[Week]'].unique()[0]
    current_week = int(current_week)

    ###getting holiday data###
    calendar_df = pd.read_csv(cwd + 'holiday1.csv')
    calendar_df = calendar_df.sort_values(['Time.[Week]'])

    calendar_df = calendar_df[calendar_df['Sales Domain.[Country]'] == country]
    calendar_df.rename(columns={'Time.[Week]': 'WEEK_NUM'}, inplace=True)

    calendar_df['WEEKStr'] = calendar_df['WEEK_NUM']

    calendar_df['WEEK_NUM'] = calendar_df['WEEK_NUM'].str.replace('-W', '')
    calendar_df = calendar_df.pivot_table(index=['WEEK_NUM', 'WEEKStr'],
                                          columns='Holiday Type',
                                          values='Is Holiday', aggfunc='mean',
                                          fill_value=0).reset_index()
    calendar_df = calendar_df.rename_axis(None, axis=1)

    for i in calendar_df.columns:
        if 'WEEK' not in i:
            calendar_df.rename(columns={i: 'holiday_' + i}, inplace=True)

    calendar_df['WEEK_NUM'] = calendar_df['WEEK_NUM'].astype(np.int64)

    cal_cols = list(calendar_df.columns)
    cal_cols.remove('WEEK_NUM')
    cal_cols.remove('WEEKStr')

    calendar_df.dtypes

    calendar_df['holidaytype'] = calendar_df[cal_cols].idxmax(axis=1)

    calendar_df['Total_Holidays'] = calendar_df[cal_cols].sum(axis=1)

    calendar_df['holidayflag'] = np.where(calendar_df['Total_Holidays'] >= 1,
                                          1, 0)

    calendar_df.drop(cal_cols, axis=1, inplace=True)
    calendar_df.drop(['Total_Holidays', 'WEEK_NUM'], axis=1, inplace=True)
    calendar_df.rename({'WEEKStr': 'WeekStr'}, axis=1, inplace=True)

    calendar_df['holidaytype'] = calendar_df['holidaytype'].str.replace(
        'holiday_', '')
    calendar_df['holidaytype'] = calendar_df['holidaytype'].str.replace(
        'Easter,Easter Monday',
        'Easter')

    calendar_df = pd.merge(calendar_df, time_df, how='left', on='WeekStr')

    ###getting segment data###
    segment_df = pd.read_csv(cwd + 'segment1.csv', dtype=str)
    segment_df = segment_df[segment_df['Sales Domain.[Sales Org]'] == salesorg]

    segment_df.rename({'Version.[Version Name]': 'Version',
                       'Sales Domain.[Sales Org]': 'Sales Org',
                       'EPM.[Category]': 'EPM_Cat',
                       'Item.[Planning Item]': 'Item'},
                      axis=1, inplace=True)

    del segment_df['Version']

    print("current_week")
    print(current_week)

    print("tran_df : {}".format(tran_df.shape))
    print("prod_df : {}".format(prod_df.shape))
    print("loc_df : {}".format(loc_df.shape))
    print("time_df : {}".format(time_df.shape))
    print("current_week: {}".format(current_week))
    print("time df head")
    print(time_df.head())
    print("calendar_df cols: {}".format(calendar_df.columns))


else:

    ###getting actual shifted values in tran_df###
    logger.debug("tran df head")
    logger.debug(tran_df.head())
    logger.debug(tran_df.dtypes)

    tran_df.rename({'Version.[Version Name]': 'Version',
                    'Sales Domain.[Sales Org]': 'Sales Org',
                    'Sales Domain.[Customer Group]': 'Customer',
                    'Location.[Location]': 'Location', 'EPM.[EPM]': 'EPM',
                    'Time.[Week]': 'WeekStr', 'Item.[L6]': 'L6',
                    'Item.[Planning Item]': 'Item', 'Actual Shifted': 'Qty'},
                   axis=1, inplace=True)

    version = tran_df['Version'].unique()[0]
    country = tran_df['Sales Org'].unique()[0][:2]
    salesorg = tran_df['Sales Org'].unique()[0]
    cat = tran_df['L6'].unique()[0]
    del tran_df['Version']
    tran_df = tran_df.astype(tran_df_dtypes)

    ###getting product master data###
    logger.debug("prod df head")
    logger.debug(prod_df.head())
    logger.debug(prod_df.dtypes)

    prod_df.rename({'Item.[Item]': 'Item', 'Item.[L3]': 'L3', 'Item.[L4]': 'L4',
                    'Item.[L5]': 'L5', 'Item.[L6]': 'L6'}, axis=1, inplace=True)
    # prod_df = prod_df.astype(prod_df_dtypes)

    ###getting location master data###
    logger.debug("loc df head")
    logger.debug(loc_df.head())
    logger.debug(loc_df.dtypes)

    loc_df.rename({'Location.[Location]': 'Location',
                   'Location.[Location Type]': 'LocType',
                   'Location.[Location Region]': 'Region'}, axis=1,
                  inplace=True)
    # loc_df = loc_df.astype(loc_df_dtypes)

    ###getting time master data###

    logger.debug("time df head")
    logger.debug(time_df.head())
    logger.debug(time_df.dtypes)

    temp = time_df.groupby('Time.[Week]').first()
    time_df = pd.merge(temp['Time.[Day]'], time_df, on='Time.[Day]', how='left')

    time_df = time_df[['Time.[Week]', 'Time.[Day]']]
    time_df['Time.[Day]'] = pd.to_datetime(time_df['Time.[Day]'],
                                           format='%d-%b-%Y')

    time_df.rename({'Time.[Week]': 'WeekStr', 'Time.[Day]': 'Week'}, axis=1,
                   inplace=True)
    time_df['Week'] = pd.to_datetime(time_df['Week'])
    time_df = time_df.drop_duplicates()
    print(time_df[time_df['WeekStr'].isna()])

    ###getting current week###
    weeklist_bt_df_1 = week_df.copy()

    weeklist_bt_df_1['Time.[Week]'] = weeklist_bt_df_1[
        'Time.[Week]'].str.replace('-W', '')

    current_week = weeklist_bt_df_1['Time.[Week]'].unique()[0]
    current_week = int(current_week)

    ###getting holiday data###
    # calendar_df = pd.read_csv(cwd + 'holiday.csv')
    calendar_df = calendar_df.sort_values(['Time.[Week]'])

    calendar_df = calendar_df[calendar_df['Sales Domain.[Country]'] == country]
    calendar_df.rename(columns={'Time.[Week]': 'WEEK_NUM'}, inplace=True)

    calendar_df['WEEKStr'] = calendar_df['WEEK_NUM']

    calendar_df['WEEK_NUM'] = calendar_df['WEEK_NUM'].str.replace('-W', '')
    calendar_df = calendar_df.pivot_table(index=['WEEK_NUM', 'WEEKStr'],
                                          columns='Holiday Type',
                                          values='Is Holiday', aggfunc='mean',
                                          fill_value=0).reset_index()
    calendar_df = calendar_df.rename_axis(None, axis=1)

    for i in calendar_df.columns:
        if 'WEEK' not in i:
            calendar_df.rename(columns={i: 'holiday_' + i}, inplace=True)

    calendar_df['WEEK_NUM'] = calendar_df['WEEK_NUM'].astype(np.int64)

    cal_cols = list(calendar_df.columns)
    cal_cols.remove('WEEK_NUM')
    cal_cols.remove('WEEKStr')

    calendar_df['holidaytype'] = calendar_df[cal_cols].idxmax(axis=1)

    calendar_df['Total_Holidays'] = calendar_df[cal_cols].sum(axis=1)

    calendar_df['holidayflag'] = np.where(calendar_df['Total_Holidays'] >= 1,
                                          1, 0)

    calendar_df.drop(cal_cols, axis=1, inplace=True)
    calendar_df.drop(['Total_Holidays', 'WEEK_NUM'], axis=1, inplace=True)
    calendar_df.rename({'WEEKStr': 'WeekStr'}, axis=1, inplace=True)

    calendar_df['holidaytype'] = calendar_df['holidaytype'].str.replace(
        'holiday_', '')
    calendar_df['holidaytype'] = calendar_df['holidaytype'].str.replace(
        'Easter,Easter Monday',
        'Easter')

    calendar_df = pd.merge(calendar_df, time_df, how='left', on='WeekStr')

    ###getting segment data###
    # segment_df = pd.read_csv(cwd + 'segment.csv', dtype=str)
    segment_df = segment_df[segment_df['Sales Domain.[Sales Org]'] == salesorg]

    segment_df.rename({'Version.[Version Name]': 'Version',
                       'Sales Domain.[Sales Org]': 'Sales Org',
                       'EPM.[Category]': 'EPM_Cat',
                       'Item.[Planning Item]': 'Item'},
                      axis=1, inplace=True)

    del segment_df['Version']

    logger.debug("current_week")
    logger.debug(current_week)

    logger.debug("tran_df : {}".format(tran_df.shape))
    logger.debug("tran df head")
    logger.debug(tran_df.head())
    logger.debug(tran_df.dtypes)

    logger.debug("prod_df : {}".format(prod_df.shape))
    logger.debug("loc_df : {}".format(loc_df.shape))
    logger.debug("time_df : {}".format(time_df.shape))
    logger.debug("Current week : {}".format(current_week))
    logger.debug("time df head")
    logger.debug(time_df.head())
    logger.debug(time_df.dtypes)
    logger.debug("calendar df")
    logger.debug(calendar_df.head())
    logger.debug("segment df")
    logger.debug(segment_df.head())

################################################################################
#####--EXECUTION--#####

###Aggregating to Sales Org level###
tran_df2 = tran_df.groupby(['Sales Org', 'Location',
                            'Item', 'WeekStr'])['Qty'].sum().reset_index()

if env != "local":
    logger.debug("tran df2: ")
    logger.debug(tran_df2.head())
    logger.debug(tran_df2.dtypes)

##Joining Week to Actuals df##
tran_df2 = pd.merge(tran_df2, time_df, how='left', on='WeekStr')
tran_df = pd.merge(tran_df, time_df, how='left', on='WeekStr')

# tran_df2 = pd.merge(tran_df2, temp_df, how = 'left', on = 'WeekStr')
# tran_df = pd.merge(tran_df, temp_df, how = 'left', on = 'WeekStr')
#
# tran_df2['holidaytype'] = tran_df2['holidaytype'].fillna(0)
# tran_df2['holidayflag'] = tran_df2['holidayflag'].fillna(0)
# tran_df['holidaytype'] = tran_df['holidaytype'].fillna(0)
# tran_df['holidayflag'] = tran_df['holidayflag'].fillna(0)
#
# holiday_col = [col for col in tran_df2 if col.startswith('holiday')]
#
# for col in holiday_col:
#     tran_df2[col+'_lead1'] = tran_df2.groupby(['Item','Location',
#                                                'Sales Org'])[col].shift(-1)
#     tran_df2[col+'_lead2'] = tran_df2.groupby(['Item','Location',
#                                                'Sales Org'])[col].shift(-2)
#     tran_df2[col+'_lead3'] = tran_df2.groupby(['Item','Location',
#                                                'Sales Org'])[col].shift(-3)
#
# tran_df2.fillna(0)

if env != "local":
    logger.debug("tran df2 after merger with time df: ")
    logger.debug(tran_df2.head())
    logger.debug(tran_df2.dtypes)

    logger.debug("tran df after merger with time df: ")
    logger.debug(tran_df.head())
    logger.debug(tran_df.dtypes)

time_df.dropna(inplace=True)
time_df['WEEK'] = time_df['WeekStr'].str.replace('-W', '')
time_df['WEEK'] = time_df['WEEK'].astype('int64')

tran_df2_orig = tran_df2.copy()
tran_df_orig = tran_df.copy()

############Start loop for backtesting here#################
max_week = time_df[time_df['WEEK'] == current_week].Week.max()
ship_max_week = current_week - 1

if env != "local":
    logger.debug("Max week for backtesting: ")
    logger.debug(max_week)

start = max_week - np.timedelta64(1, 'W')

if env != "local":
    logger.debug("start: ")
    logger.debug(start)

pd.options.mode.chained_assignment = None  # default='warn'

# for c in range(1, cycles + 1):
#     c = 1

hist_week = start
if env != "local":
    logger.debug("hist week: ")
    logger.debug(hist_week)

# if env == "local":
#     print(str(c) + " out of " + str(cycles) + " : " + str(hist_week))
# else:
#     logger.debug(str(c) + " out of " + str(cycles) + " : " + str(hist_week))

####Filtering out data per the bcktest cycle####
tran_df2 = tran_df2_orig[tran_df2_orig['Week'] <= hist_week]
tran_df = tran_df_orig[tran_df_orig['Week'] <= hist_week]

## Filtering out SKU with 0 volume in last 56 weeks ##
EOL_Date = tran_df2['Week'].max() - np.timedelta64(7 * 56, 'D')
Valid_SKUs = tran_df2[tran_df2['Week'] > EOL_Date]
Valid_SKUs = Valid_SKUs.groupby(['Sales Org', 'Location', 'Item'])[
    'Qty'].sum().reset_index()
Valid_SKUs = Valid_SKUs[Valid_SKUs['Qty'] > 0]
Valid_SKUs['Key'] = Valid_SKUs['Sales Org'] + '-' + Valid_SKUs[
    'Location'] + '-' + Valid_SKUs['Item']

if env != "local":
    logger.debug("Valid_SKUs : {}".format(Valid_SKUs.shape))

tran_df2['Key'] = tran_df2['Sales Org'] + '-' + tran_df2['Location'] + '-' + \
                  tran_df2['Item']
tran_df2 = tran_df2[tran_df2['Key'].isin(Valid_SKUs['Key'])]
del tran_df2['Key']

if env != "local":
    logger.debug("tran_df2 : {}".format(tran_df2.shape))
    logger.debug(tran_df2.head())

#### Aggregating filling holes #####
df1 = tran_df2.groupby(['Sales Org', 'Location', 'Item', 'Week'])[
    'Qty'].sum()

if env != "local":
    logger.debug("df1 : {}".format(df1.shape))
    logger.debug(df1.head())

## Filling holes by pivot melt. Can also do stack/unstack or reindex ##

df1_StartingDate = tran_df2.groupby(['Sales Org', 'Location', 'Item'])[
    'Week'].min().reset_index()
df1_StartingDate = df1_StartingDate.rename(
    columns={'Week': 'Starting_Week'})

if env != "local":
    logger.debug("df1_StartingDate : {}".format(df1_StartingDate.shape))
    logger.debug(df1_StartingDate.head())

df1 = df1.unstack().stack(dropna=False).fillna(0).astype(int)
df1 = df1.reset_index()
df1 = df1.rename(columns={0: 'Qty'})

df1 = pd.merge(df1, df1_StartingDate, how='left',
               on=['Sales Org', 'Location', 'Item'])
df1 = df1[df1['Week'] >= df1['Starting_Week']]
# df1.info()

if env != "local":
    logger.debug("df1 after stack unstack")
    logger.debug(df1.shape)

########Joining#######
## Joining to get dimension attributes ##
df1 = pd.merge(df1, prod_df, how='left', on='Item')
df1 = pd.merge(df1, loc_df, how='left', on='Location')

###Creating Item_Loc Key###
df1['Item_Loc'] = df1['Item'] + '_' + df1['Location']

if env != "local":
    logger.debug("df1 after joining the dimension attributes")
    logger.debug(df1.shape)


##### Feature Engineering #####

### Creating Time Features ###
def time_features(df):
    df['Year_Nominal'] = df['Week'].dt.year
    df['Year_Num'] = df['Year_Nominal'] - df['Year_Nominal'].min() + 1
    # df['Year_Nominal'] = df['Year_Nominal'].astype(str)

    df['Quarter_Num'] = df['Week'].dt.quarter
    df['Quarter_Sin'] = np.sin(2 * np.pi * (df['Quarter_Num'] - 1) / 4)
    df['Quarter_Cos'] = np.cos(2 * np.pi * (df['Quarter_Num'] - 1) / 4)
    # df['Quarter_Nominal'] = df['Quarter_Num'].astype(str)

    df['Month_Num'] = df['Week'].dt.month
    df['Month_Sin'] = np.sin(2 * np.pi * (df['Month_Num'] - 1) / 12)
    df['Month_Cos'] = np.cos(2 * np.pi * (df['Month_Num'] - 1) / 12)
    df['Month_Count'] = ((df['Week'] - df['Week'].min()) /
                         np.timedelta64(1, 'M')).astype(int)
    # df['Month_Nominal'] = df['Month_Num'].astype(str)

    df['Week_Num'] = df['Week'].dt.week
    df['Week_Sin'] = np.sin(2 * np.pi * (df['Month_Num'] - 1) / 52)
    df['Week_Cos'] = np.cos(2 * np.pi * (df['Month_Num'] - 1) / 52)
    # df['Week_Nominal'] = df['Week_Num'].astype(str)
    df['WOM_Num'] = df['Week'].apply(lambda d: (d.day - 1) // 7 + 1)
    # df['WOM_Nominal'] = df['WOM_Num'].astype(str)

    df['Year_End'] = np.where(df['Week_Num'] >= 51, 1, 0)
    # temp = df1[['Year_End', 'Day', 'Christmas', 'Holiday']]

    if env != "local":
        logger.debug("df after creating time features")
        logger.debug(df.shape)

    return df


df1 = time_features(df1)


######Creating aggregated POS features########
def agg_features(df):
    df = df.join(df.groupby(['Item', 'Week'])['Qty'].mean(),
                 on=['Item', 'Week'], rsuffix='_Item')
    df = df.join(df.groupby(['Location', 'Week'])['Qty'].mean(),
                 on=['Location', 'Week'], rsuffix='_Location')
    df = df.join(df.groupby(['L5', 'Week'])['Qty'].mean(),
                 on=['L5', 'Week'], rsuffix='_L5')
    df = df.join(df.groupby(['LocType', 'Week'])['Qty'].mean(),
                 on=['LocType', 'Week'], rsuffix='_LocType')

    if env != "local":
        logger.debug("df after creating aggregated POS features")
        logger.debug(df.shape)

    return (df)


df1 = agg_features(df1)


######Creating lag variables########
def lag_features_fn(df, col_index, col_time, col_shift, lag_start=1,
                    lag_end=9, ly_start=51, ly_end=53, lly_start=103,
                    lly_end=105):
    # df = test_df
    Data_For_Shift = df[col_index + col_time + col_shift]
    Data_For_Shift = Data_For_Shift.set_index(col_index + col_time)

    Data_Shifted = df[col_index + col_time]
    Data_Shifted = Data_Shifted.set_index(col_index + col_time)

    for i in range(lag_start, lag_end + 1):
        Data_Shift = Data_For_Shift.groupby(level=col_index).shift(i)
        Data_Shifted = Data_Shifted.join(
            Data_Shift.rename(columns=lambda x: x + "_Lag" + str(i)))

    for i in range(ly_start, ly_end + 1):
        Data_Shift = Data_For_Shift.groupby(level=col_index).shift(i)
        Data_Shifted = Data_Shifted.join(
            Data_Shift.rename(columns=lambda x: x + "_Lag" + str(i)))

    for i in range(lly_start, lly_end + 1):
        Data_Shift = Data_For_Shift.groupby(level=col_index).shift(i)
        Data_Shifted = Data_Shifted.join(
            Data_Shift.rename(columns=lambda x: x + "_Lag" + str(i)))

    Data_Shifted = Data_Shifted.reset_index()
    Data_Shifted = Data_Shifted.reindex(sorted(Data_Shifted.columns),
                                        axis=1)

    df = pd.merge(df, Data_Shifted, how='left', on=(col_index + col_time))

    if env != "local":
        logger.debug("df after creating lag features")
        logger.debug(df.shape)

    return (df)


col_shift = ['Qty', 'Qty_Item', 'Qty_Location', 'Qty_L5', 'Qty_LocType']
col_index = ['Sales Org', 'Location', 'Item']
col_time = ['Week']
lag_start = 4
lag_end = 9
ly_start = 51
ly_end = 53
lly_start = 103
lly_end = 105

df2 = lag_features_fn(df1, col_index, col_time, col_shift, lag_start, lag_end,
                      ly_start, ly_end, lly_start, lly_end)


# def ly_fill(df, col, ly_start, ly_end, ly_shift = 52):
#     for i in range(int(ly_start), int(ly_end+1)):
#         df[col+'_Lag'+str(i+ly_shift)].fillna(df[col+'_Lag'+str(i)], inplace=True)

# ly_fill(df2, 'Qty', ly_start, ly_end, 52)
# ly_fill(df2, 'Qty_Item', ly_start, ly_end, 52)
# ly_fill(df2, 'Qty_Location', ly_start, ly_end, 52)
# ly_fill(df2, 'Qty_L5', ly_start, ly_end, 52)
# ly_fill(df2, 'Qty_LocType', ly_start, ly_end, 52)


def Lag_Indices_MA_diff(Lag_df, Lag_Measure, Lag_Start, Lag_End, LY_Start,
                        LY_End, LLY_Start, LLY_End):
    Lag_Start_Loc = Lag_df.columns.get_loc(
        str(Lag_Measure) + "_Lag" + str(Lag_Start))
    Lag_End_Loc = Lag_df.columns.get_loc(
        str(Lag_Measure) + "_Lag" + str(Lag_End)) + 1
    Lag_df[Lag_Measure + "_" + str(Lag_Start) + "_" + str(
        Lag_End) + "_MA"] = Lag_df.iloc[:, Lag_Start_Loc:Lag_End_Loc].mean(
        axis=1, skipna=True)
    LY_Start_Loc = Lag_df.columns.get_loc(
        str(Lag_Measure) + "_Lag" + str(LY_Start))
    LY_End_Loc = Lag_df.columns.get_loc(
        str(Lag_Measure) + "_Lag" + str(LY_End)) + 1
    Lag_df[Lag_Measure + "_" + str(LY_Start) + "_" + str(
        LY_End) + "_MA"] = Lag_df.iloc[:, LY_Start_Loc:LY_End_Loc].mean(
        axis=1, skipna=True)
    LLY_Start_Loc = Lag_df.columns.get_loc(
        str(Lag_Measure) + "_Lag" + str(LLY_Start))
    LLY_End_Loc = Lag_df.columns.get_loc(
        str(Lag_Measure) + "_Lag" + str(LLY_End)) + 1
    Lag_df[Lag_Measure + "_" + str(LLY_Start) + "_" + str(
        LLY_End) + "_MA"] = Lag_df.iloc[:, LLY_Start_Loc:LLY_End_Loc].mean(
        axis=1, skipna=True)
    Lag_df[Lag_Measure + "_" + str(Lag_Start) + "_" + str(
        Lag_End) + "_trend"] = 0
    for i in range(int(Lag_Start), int(Lag_End)):
        Lag_df[Lag_Measure + "_" + str(i) + "_" + str(i + 1) + "_diff"] = \
            Lag_df[Lag_Measure + "_Lag" + str(i)] - Lag_df[
                Lag_Measure + "_Lag" + str(i + 1)]
        Lag_df[Lag_Measure + "_" + str(Lag_Start) + "_" + str(
            Lag_End) + "_trend"] = Lag_df[Lag_Measure + "_" + str(
            i) + "_" + str(i + 1) + "_diff"].fillna(0) + Lag_df[
                                       Lag_Measure + "_" + str(
                                           Lag_Start) + "_" + str(
                                           Lag_End) + "_trend"]


Lag_Indices_MA_diff(df2, "Qty", lag_start, lag_end, ly_start, ly_end,
                    lly_start, lly_end)
Lag_Indices_MA_diff(df2, "Qty_Location", lag_start, lag_end, ly_start,
                    ly_end, lly_start, lly_end)
Lag_Indices_MA_diff(df2, "Qty_Item", lag_start, lag_end, ly_start, ly_end,
                    lly_start, lly_end)
Lag_Indices_MA_diff(df2, "Qty_L5", lag_start, lag_end, ly_start, ly_end,
                    lly_start, lly_end)
Lag_Indices_MA_diff(df2, "Qty_LocType", lag_start, lag_end, ly_start,
                    ly_end, lly_start, lly_end)

# def seasonal_lag_feat(Lag_df, Lag_Measure, Lag_Start = 51, Lag_End = 53):
#     #Backfill lag24 and lag 36 if needed

#     Lag_Start_Loc = Lag_df.columns.get_loc(str(Lag_Measure)+"_Lag"+str(Lag_Start))
#     Lag_End_Loc = Lag_df.columns.get_loc(str(Lag_Measure)+"_Lag"+str(Lag_End)) + 1
#     Lag_df[Lag_Measure+"_"+str(Lag_Start)+"_"+str(Lag_End)+"_diff"] = \
#         Lag_df[Lag_Measure+"_Lag"+str(Lag_Start)] - Lag_df[Lag_Measure+"_Lag"+str(Lag_End)]
#     Lag_df[Lag_Measure+"_"+str(Lag_Start)+"_"+str(Lag_End)+"_perc"] = \
#         Lag_df[Lag_Measure+"_Lag"+str(Lag_Start)] / Lag_df[Lag_Measure+"_Lag"+str(Lag_End)]
#     Lag_df[Lag_Measure+"_"+str(Lag_Start)+"_"+str(Lag_End)+"_mean"] = \
#         Lag_df[[Lag_Measure+"_Lag"+str(Lag_Start), Lag_Measure+"_Lag"+str(Lag_End)]].mean(axis=1)


# seasonal_lag_feat(df2,"Qty",51,53)
# seasonal_lag_feat(df2,"Qty_Location",51,53)
# seasonal_lag_feat(df2,"Qty_Item",51,53)
# seasonal_lag_feat(df2,"Qty_L5",51,53)
# seasonal_lag_feat(df2,"Qty_LocType",51,53)

# seasonal_lag_feat(df2,"Qty",103,105)
# seasonal_lag_feat(df2,"Qty_Location",103,105)
# seasonal_lag_feat(df2,"Qty_Item",103,105)
# seasonal_lag_feat(df2,"Qty_L5",103,105)
# seasonal_lag_feat(df2,"Qty_LocType",103,105)


####### Model Training, Tuning and Predicting #######


df2 = pd.merge(df2, calendar_df, how='left', on='Week')

df2['holidaytype'] = df2['holidaytype'].fillna(0)
df2['holidayflag'] = df2['holidayflag'].fillna(0)

holiday_col = [col for col in df2 if col.startswith('holiday')]

for col in holiday_col:
    df2[col + '_lead1'] = df2.groupby(['Item', 'Location',
                                       'Sales Org'])[col].shift(-1)
    df2[col + '_lead2'] = df2.groupby(['Item', 'Location',
                                       'Sales Org'])[col].shift(-2)
    df2[col + '_lead3'] = df2.groupby(['Item', 'Location',
                                       'Sales Org'])[col].shift(-3)

df2['holidaytype_lead1'] = df2['holidaytype_lead1'].fillna(0)
df2['holidayflag_lead1'] = df2['holidayflag_lead1'].fillna(0)
df2['holidaytype_lead2'] = df2['holidaytype_lead2'].fillna(0)
df2['holidayflag_lead2'] = df2['holidayflag_lead2'].fillna(0)
df2['holidaytype_lead3'] = df2['holidaytype_lead3'].fillna(0)
df2['holidayflag_lead3'] = df2['holidayflag_lead3'].fillna(0)

####Concat Item_Location#####
df2['Item_Loc'] = df2['Item'] + '_' + df2['Location']

###### Categorical feature encoding #######
Item_Loc_be = pd.DataFrame(df2['Item_Loc'].drop_duplicates())
encoder = ce.BinaryEncoder(cols='Item_Loc', return_df=True)
Item_Loc_be = pd.concat([Item_Loc_be, encoder.fit_transform(Item_Loc_be)],
                        axis=1)
enc_cols = Item_Loc_be.iloc[:, 1:].columns.to_list()
df2 = pd.merge(df2, Item_Loc_be, how='left', on='Item_Loc')

encoder = ce.TargetEncoder(cols='Item_Loc', return_df=True)
Item_Loc_te = pd.concat(
    [df2['Item_Loc'], encoder.fit_transform(df2['Item_Loc'], df2['Qty'])],
    axis=1)
Item_Loc_te.columns = ['Item_Loc', 'Item_Loc_te']
Item_Loc_te = pd.DataFrame(
    Item_Loc_te[['Item_Loc', 'Item_Loc_te']].drop_duplicates())
enc_cols = enc_cols + [Item_Loc_te.columns[1]]
df2 = pd.merge(df2, Item_Loc_te, how='left', on='Item_Loc')

###################Creating train df################################
## Only filtering 36 months for training ##
Train_Periods = 37
Start_Time = hist_week - pd.DateOffset(months=Train_Periods)

train_df = df2[df2['Week'] >= Start_Time]
train_df['Lag'] = np.nan

if env != "local":
    logger.debug("train_df shape")
    logger.debug(train_df.shape)

##########Generating sample weights#########

###Generating row weights based on recency and seasonality###
row_weights = train_df['Week']
row_weights.drop_duplicates(inplace=True)
row_weights = pd.DataFrame(row_weights).sort_values('Week').reset_index()
row_weights['interval'] = hist_week + np.timedelta64(1, 'W') - row_weights[
    'Week']
row_weights['interval'] = (row_weights['interval'] / np.timedelta64(1,
                                                                    'W')) - 1
row_weights['weight'] = 1 * np.exp(-row_weights['interval'] * (0.0075))

# Add seasonal weights ly
seas_start = hist_week - np.timedelta64(7 * 54, 'D')
seas_end = hist_week - np.timedelta64(7 * 50, 'D')
seas_range = pd.date_range(start=seas_start, end=seas_end, freq='W-SUN')
row_weights['weight'] = np.where(row_weights['Week'].isin(seas_range),
                                 0.8, row_weights['weight'])

# Add seasonal weights lly
seas_start = hist_week - np.timedelta64(7 * 105, 'D')
seas_end = hist_week - np.timedelta64(7 * 101, 'D')
seas_range = pd.date_range(start=seas_start, end=seas_end, freq='W-SUN')
row_weights['weight'] = np.where(row_weights['Week'].isin(seas_range),
                                 0.7, row_weights['weight'])

# Reduce weight during covid period
seas_start = pd.to_datetime('2020-02-01')
seas_end = pd.to_datetime('2020-07-01')
seas_range = pd.date_range(start=seas_start, end=seas_end, freq='W-SUN')
row_weights['weight'] = np.where(row_weights['Week'].isin(seas_range),
                                 0.1, row_weights['weight'])

# Reduce weight for 2020
seas_start = pd.to_datetime('2020-09-01')
seas_end = pd.to_datetime('2021-02-01')
seas_range = pd.date_range(start=seas_start, end=seas_end, freq='W-SUN')
row_weights['weight'] = np.where(row_weights['Week'].isin(seas_range),
                                 0.3, row_weights['weight'])

# train_df_orig = train_df_orig.merge(row_weights, how = 'left', on = 'Day')
train_df = pd.merge(train_df.copy(), row_weights[['Week', 'weight']],
                    how='left', on='Week')

###joining segment df###

train_df = pd.merge(train_df, segment_df, how='left', on=['Sales Org', 'Item'])

if env != "local":
    logger.debug("train_df shape after adding weights")
    logger.debug(train_df.shape)

train_df['Month_Count'] = train_df['Month_Count'] + 21

lead1list = ['Qty_Item_Lag103',
'Qty_Item_Lag51',
'Qty_L5_Lag103',
'Qty_L5_Lag51',
'Qty_Lag103',
'Qty_Lag51',
'Qty_LocType_Lag103',
'Qty_LocType_Lag51',
'Qty_Location_Lag103',
'Qty_Location_Lag51',
]

for col in lead1list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-1)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'],inplace=True,axis=1)

lead43list = ['Qty_Item_Lag9',
'Qty_L5_Lag9',
'Qty_Lag9',
'Qty_LocType_Lag9',
'Qty_Location_Lag9',
]

for col in lead43list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-43)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)

lead44list = ['Qty_Item_Lag8',
'Qty_L5_Lag8',
'Qty_Lag8',
'Qty_LocType_Lag8',
'Qty_Location_Lag8',

]

for col in lead44list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-44)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)

lead45list = ['Qty_Item_Lag7',
'Qty_L5_Lag7',
'Qty_Lag7',
'Qty_LocType_Lag7',
'Qty_Location_Lag7',
]

for col in lead45list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-45)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)

lead46list = ['Qty_Item_Lag6',
'Qty_L5_Lag6',
'Qty_Lag6',
'Qty_LocType_Lag6',
'Qty_Location_Lag6',
]

for col in lead46list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-46)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)

lead47list = ['Qty_Item_Lag5',
'Qty_L5_Lag5',
'Qty_Lag5',
'Qty_LocType_Lag5',
'Qty_Location_Lag5',
]

for col in lead47list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-47)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)

lead48list = ['Qty_Item_Lag4',
'Qty_L5_Lag4',
'Qty_Lag4',
'Qty_LocType_Lag4',
'Qty_Location_Lag4',
]

for col in lead48list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(-48)
    train_df[col] = np.where(train_df[col].isna(),
                             train_df[col + '_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)

lead0list = ['Qty_Item_Lag104',
'Qty_Item_Lag105',
'Qty_Item_Lag52',
'Qty_Item_Lag53',
'Qty_L5_Lag104',
'Qty_L5_Lag105',
'Qty_L5_Lag52',
'Qty_L5_Lag53',
'Qty_Lag104',
'Qty_Lag105',
'Qty_Lag52',
'Qty_Lag53',
'Qty_LocType_Lag104',
'Qty_LocType_Lag105',
'Qty_LocType_Lag52',
'Qty_LocType_Lag53',
'Qty_Location_Lag104',
'Qty_Location_Lag105',
'Qty_Location_Lag52',
'Qty_Location_Lag53',
]

for col in lead0list:
    train_df[col+'_new'] = train_df.groupby(['Sales Org','Location', 'Item'])['Qty'].shift(0)
    train_df[col] = np.where(train_df[col].isna(), train_df[col+'_new'], train_df[col])
    train_df.drop([col + '_new'], inplace=True, axis=1)


###################Creating prediction df################################
#### Create Test dataset ####
Valid_SKUs = train_df[
    train_df['Week'] > hist_week - np.timedelta64(31, 'D')]
Valid_SKUs = Valid_SKUs.groupby(['Sales Org', 'Location', 'Item'])[
    'Qty'].sum().reset_index()
Valid_SKUs = Valid_SKUs[Valid_SKUs['Qty'] > 0]
del Valid_SKUs['Qty']

Fcst_End = hist_week + np.timedelta64(f_wk, 'W')
Fcst_Start = hist_week + np.timedelta64(1, 'W')
Forecast_Range = pd.date_range(start=Fcst_Start, end=Fcst_End, freq='W-SUN')
Forecast_Range = pd.DataFrame(Forecast_Range)
Forecast_Range['Lag'] = Forecast_Range.index - 1

# Merging to create test df
Valid_SKUs['key'] = 0
Forecast_Range['key'] = 0
test_df = Valid_SKUs.merge(Forecast_Range, how='outer', on='key')
del test_df['key']
test_df.columns = ['Sales Org', 'Location', 'Item', 'Week', 'Lag']
test_df['Item'] = test_df['Item'].astype('str')
test_df['Location'] = test_df['Location'].astype('str')
test_df['Qty'] = np.nan

if env != "local":
    logger.debug("test_df shape after creating test dataset")
    logger.debug(test_df.shape)

### Appending with train df ###

test_df = test_df.append(train_df[test_df.columns]).sort_values(
    ['Sales Org', 'Location', 'Item', 'Week'], ascending=True)

if env != "local":
    logger.debug("test_df shape after appending with train_df")
    logger.debug(test_df.shape)

##Repeat dataframe creation steps##
test_df = pd.merge(test_df, prod_df, how='left', on='Item')
test_df = pd.merge(test_df, loc_df, how='left', on='Location')
###Creating Item_Loc Key###
test_df['Item_Loc'] = test_df['Item'] + '_' + test_df['Location']

test_df = time_features(test_df)
test_df = agg_features(test_df)
test_df = lag_features_fn(test_df, col_index, col_time, col_shift,
                          lag_start, lag_end, ly_start, ly_end, lly_start,
                          lly_end)

# ly_fill(test_df, 'Qty', ly_start, ly_end, 52)
# ly_fill(test_df, 'Qty_Item', ly_start, ly_end, 52)
# ly_fill(test_df, 'Qty_Location', ly_start, ly_end, 52)
# ly_fill(test_df, 'Qty_L5', ly_start, ly_end, 52)
# ly_fill(test_df, 'Qty_LocType', ly_start, ly_end, 52)


Lag_Indices_MA_diff(test_df, "Qty", lag_start, lag_end, ly_start, ly_end,
                    lly_start, lly_end)
Lag_Indices_MA_diff(test_df, "Qty_Location", lag_start, lag_end, ly_start,
                    ly_end, lly_start, lly_end)
Lag_Indices_MA_diff(test_df, "Qty_Item", lag_start, lag_end, ly_start,
                    ly_end, lly_start, lly_end)
Lag_Indices_MA_diff(test_df, "Qty_L5", lag_start, lag_end, ly_start, ly_end,
                    lly_start, lly_end)
Lag_Indices_MA_diff(test_df, "Qty_LocType", lag_start, lag_end, ly_start,
                    ly_end, lly_start, lly_end)

# seasonal_lag_feat(df2,"Qty",51,53)
# seasonal_lag_feat(df2,"Qty_Location",51,53)
# seasonal_lag_feat(df2,"Qty_Item",51,53)
# seasonal_lag_feat(df2,"Qty_L5",51,53)
# seasonal_lag_feat(df2,"Qty_LocType",51,53)

# seasonal_lag_feat(df2,"Qty",103,105)
# seasonal_lag_feat(df2,"Qty_Location",103,105)
# seasonal_lag_feat(df2,"Qty_Item",103,105)
# seasonal_lag_feat(df2,"Qty_L5",103,105)
# seasonal_lag_feat(df2,"Qty_LocType",103,105)


test_df = pd.merge(test_df, calendar_df, how='left', on='Week')

test_df['holidaytype'] = test_df['holidaytype'].fillna(0)
test_df['holidayflag'] = test_df['holidayflag'].fillna(0)

holiday_col = [col for col in test_df if col.startswith('holiday')]

for col in holiday_col:
    test_df[col + '_lead1'] = test_df.groupby(['Item', 'Location',
                                               'Sales Org'])[col].shift(-1)
    test_df[col + '_lead2'] = test_df.groupby(['Item', 'Location',
                                               'Sales Org'])[col].shift(-2)
    test_df[col + '_lead3'] = test_df.groupby(['Item', 'Location',
                                               'Sales Org'])[col].shift(-3)

test_df['holidaytype_lead1'] = test_df['holidaytype_lead1'].fillna(0)
test_df['holidayflag_lead1'] = test_df['holidayflag_lead1'].fillna(0)
test_df['holidaytype_lead2'] = test_df['holidaytype_lead2'].fillna(0)
test_df['holidayflag_lead2'] = test_df['holidayflag_lead2'].fillna(0)
test_df['holidaytype_lead3'] = test_df['holidaytype_lead3'].fillna(0)
test_df['holidayflag_lead3'] = test_df['holidayflag_lead3'].fillna(0)

###joining segment df to test_df###

test_df = pd.merge(test_df, segment_df, how='left', on=['Sales Org', 'Item'])

test_df = pd.merge(test_df, Item_Loc_be, how='left', on='Item_Loc')
test_df = pd.merge(test_df, Item_Loc_te, how='left', on='Item_Loc')

##Filtering Lag1 Month##
test_df_lr = test_df.copy()
test_df = test_df[test_df['Lag'] == 2]


###################Training####################
#####Feature_Imp_cols#####
def get_feature_imp(algo, algo_name):
    algo = lgb
    importance = algo.feature_importances_.round(4)
    names = np.array(feature_cols)
    Feature_Imp = pd.DataFrame(data=np.column_stack((names, importance)
                                                    ),
                               columns=['names', 'importance'])
    Feature_Imp.columns = ['Feature', 'Feature_Importance']
    Feature_Imp['Week'] = hist_week
    Feature_Imp = Feature_Imp[['Week', 'Feature', 'Feature_Importance']]
    Feature_Imp = Feature_Imp.rename(
        columns={'Feature_Importance': algo_name})
    return (Feature_Imp)


########Model1: Predicting the lag 2 week###############

ts_cols = ['Year_Nominal', 'Year_Num',
           'Quarter_Num', 'Quarter_Sin', 'Quarter_Cos', 'Month_Num',
           'Month_Sin',
           'Month_Cos', 'Month_Count', 'Week_Num', 'Week_Sin', 'Week_Cos',
           'WOM_Num', 'Year_End',
           'Qty_Lag4', 'Qty_Lag5', 'Qty_Lag6', 'Qty_Lag7', 'Qty_Lag8',
           'Qty_Lag9',
           'Qty_Lag51', 'Qty_Lag52', 'Qty_Lag53', 'Qty_Lag104',
           'Qty_Lag105',
           'Qty_4_9_MA', 'Qty_51_53_MA', 'Qty_4_9_trend', 'Qty_4_5_diff',
           'Qty_5_6_diff', 'Qty_6_7_diff',
           'Qty_Item_Lag4', 'Qty_Item_Lag5', 'Qty_Item_Lag6',
           'Qty_Item_Lag52',
           'Qty_Item_4_9_MA', 'Qty_Item_51_53_MA', 'Qty_Item_4_9_trend',
           'Qty_Item_4_5_diff',
           'Qty_Location_Lag4', 'Qty_Location_Lag5', 'Qty_Location_Lag6',
           'Qty_Location_Lag52',
           'Qty_Location_4_9_MA', 'Qty_Location_51_53_MA',
           'Qty_Location_4_9_trend', 'Qty_Location_4_5_diff',
           'Qty_L5_Lag4', 'Qty_L5_Lag5', 'Qty_L5_Lag6', 'Qty_L5_Lag52',
           'Qty_L5_4_9_MA', 'Qty_L5_51_53_MA', 'Qty_L5_4_9_trend',
           'Qty_L5_4_5_diff', 'holidayflag_lead1', 'holidayflag_lead2',
           'holidayflag_lead3']

cat_cols = ['Item', 'L3', 'L5', 'LocType', 'Region', 'holidaytype_lead1',
            'holidaytype_lead2', 'holidaytype_lead3', 'Product Segment']

feature_cols = ts_cols + cat_cols

train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].astype("category")

####Creating copies of original test and train df###
# train_df_orig = train_df.copy()
# test_df_orig = test_df.copy()
feature_cols_orig = feature_cols

######################Slicing the model######################
# #By Segment Group

# train_df_orig['Slice'] = train_df_orig['L6']
# test_df_orig['Slice'] = test_df_orig['L6']

# train_df_orig['Slice'] = train_df_orig['L1'].astype(str) + "_" + train_df_orig['Group'].astype(str)
# test_df_orig['Slice'] = test_df_orig['L1'].astype(str) + "_" + test_df_orig['Group'].astype(str)

# train_df_orig['Slice'] = train_df_orig['Location'].astype(str) + "_" + train_df_orig['Group'].astype(str)
# test_df_orig['Slice'] = test_df_orig['Location'].astype(str) + "_" + test_df_orig['Group'].astype(str)

# iter_list = train_df_orig['Slice'].unique()
# iter_count = iter_list.size + 1

# for j in range(1, iter_count):
#     # j=1
try:
    # if env == "local":
    #     print(str(j) + " out of " + str(iter_count - 1) + " : " + iter_list[
    #     j - 1])
    # else:
    #     logger.debug(str(j) + " out of " + str(iter_count - 1) + " : " +
    #                  iter_list[
    #                      j - 1])

    # train_df = train_df_orig.copy()

    if env != "local":
        logger.debug("model run starting")
    else:
        print("model run starting")

    ###XGB RF##
    ##Select feature set##
    feature_cols = ts_cols
    ## Creating the Training matrix with all feature columns##
    X = train_df.loc[:, feature_cols]
    ## Creating response vector
    y = train_df['Qty']
    # xgb_rf = XGBRFRegressor(random_state=42)
    if env != "local":
        logger.debug("xgb_rf model run starting")
    else:
        print("xgb_rf model run starting")
    xgb_rf = XGBRFRegressor(n_estimators=500, random_state=42)
    # fit the model
    start_time = time.time()
    xgb_rf.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("xgb_rf --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("xgb_rf --- %s seconds ---" % (time.time() -
                                                    start_time))

    # Feature_Imp_all = get_feature_imp(xgb_rf,'xgb_rf')
    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(xgb_rf,'xgb_rf'),
    #                         how = 'left', on = ['Time','Feature'])

    ###xgb##
    if env != "local":
        logger.debug("xgb model run starting")
    else:
        print("xgb model run starting")
    xgb = XGBRegressor(n_estimators=500, learning_rate=0.1,
                       random_state=42)
    # fit the model
    start_time = time.time()
    xgb.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("xgb --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("xgb --- %s seconds ---" % (time.time() - start_time))

    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(xgb,'xgb'),
    #                   how = 'left', on = ['Week','Feature'])

    ###LGBM##
    ##Select feature set##
    feature_cols = feature_cols_orig
    ## Creating the Training matrix with all feature columns##
    train_df = train_df.dropna(subset=['Product Segment'])

    X = train_df.loc[:, feature_cols]
    ## Creating response vector
    y = train_df['Qty']
    if env != "local":
        logger.debug("lgbm model run starting")
    else:
        print("lgbm model run starting")
    lgb = LGBMRegressor(n_estimators=2000, learning_rate=0.005,
                        max_depth=10, num_leaves=int((2 ** 10) / 2),
                        max_bin=1000,
                        random_state=42)
    # lgb = LGBMRegressor(random_state=42, verbose = -1)
    # fit the model
    start_time = time.time()
    lgb.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("lgb --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("lgb --- %s seconds ---" % (time.time() - start_time))

    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(lgb,'lgb'),
    #                   how = 'left', on = ['Week','Feature'])

    ###Catboost##
    # cb = CatBoostRegressor(n_estimators=5000, learning_rate = 0.005,
    #                        # cat_features = cat_cols, one_hot_max_size = 16,
    #                        random_state=42, verbose=0)
    if env != "local":
        logger.debug("catb model run starting")
    else:
        print("catb model run starting")
    cb = CatBoostRegressor(n_estimators=1000, learning_rate=0.01,
                           cat_features=cat_cols, one_hot_max_size=16,
                           random_state=42, verbose=0)
    # fit the model
    start_time = time.time()
    cb.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("cb --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("cb --- %s seconds ---" % (time.time() - start_time))

    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(cb,'cb'),
    #                   how = 'left', on = ['Time','Feature'])

    ########################Predicting########################
    # test_df = test_df_orig.copy()

    #####Prediction#####
    feature_cols = ts_cols
    y_pred = xgb_rf.predict(test_df.loc[:, feature_cols])
    test_df['xgb_rf'] = y_pred
    test_df['xgb_rf'][test_df['xgb_rf'] < 0] = 0

    y_pred = xgb.predict(test_df.loc[:, feature_cols])
    test_df['xgb'] = y_pred
    test_df['xgb'][test_df['xgb'] < 0] = 0

    feature_cols = feature_cols_orig
    y_pred = lgb.predict(test_df.loc[:, feature_cols])
    test_df['lgb'] = y_pred
    test_df['lgb'][test_df['lgb'] < 0] = 0

    y_pred = cb.predict(test_df.loc[:, feature_cols])
    test_df['cb'] = y_pred
    test_df['cb'][test_df['cb'] < 0] = 0

    # # Appending Feature Imp
    # if j == 1:
    #     Feature_Imp_df = Feature_Imp_all
    # else:
    #     Feature_Imp_df = Feature_Imp_df.append(Feature_Imp_all)

    # Appending test_df
    # if j == 1:
    #     test_df_all = test_df
    # else:
    #     test_df_all = test_df_all.append(test_df)

    # test_df_all = test_df.copy()

    if env == "local":
        print("test_df")
        print(test_df.head(10))
    else:
        logger.debug("test_df")
        logger.debug(test_df.head(10))
        logger.debug("test df ")
        logger.debug(test_df.dtypes)
        import datetime

        t = datetime.date(2022, 11, 6)

        logger.debug(test_df[(test_df['Week'] == t) & (
                    test_df['Item'] == '000000447000198300') & (
                                         test_df['Location'] == '0457')])

except:
    pass

####Outputting####
output_df_l2 = test_df[['Sales Org', 'Location', 'Item', 'Week', 'Qty',
                        'xgb_rf', 'xgb', 'lgb', 'cb']]

if env != "local":
    logger.debug("Shape of output df l2 before final format")
    logger.debug(output_df_l2.shape)
    logger.debug("output df l2 before final format")
    logger.debug(output_df_l2.head())
    logger.debug(output_df_l2.dtypes)
    logger.debug("intersection op df l2")
    import datetime

    t = datetime.date(2022, 11, 6)
    logger.debug(output_df_l2[(output_df_l2['Week'] == t) & (
                output_df_l2['Item'] == '000000447000198300') & (
                                          output_df_l2['Location'] == '0457')])

########Model2: Predicting long range (lr) forecast###############

ts_cols = ['Year_Nominal', 'Year_Num',
           'Quarter_Num', 'Quarter_Sin', 'Quarter_Cos', 'Month_Num',
           'Month_Sin',
           'Month_Cos', 'Month_Count', 'Week_Num', 'Week_Sin', 'Week_Cos',
           'WOM_Num', 'Year_End',
           'holidayflag_lead1', 'holidayflag_lead2',
           'holidayflag_lead3']

cat_cols = ['Item_Loc', 'L3', 'LocType', 'holidaytype_lead1',
            'holidaytype_lead2', 'holidaytype_lead3', 'Product Segment']

feature_cols = ts_cols + enc_cols + cat_cols
feature_cols_orig = feature_cols

train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df_lr[cat_cols] = test_df_lr[cat_cols].astype("category")

# Copy of test dataframe
# test_df_lr_orig = test_df_lr.copy()

######################Slicing the model######################
# #By Segment Group

# train_df_orig['Slice'] = train_df_orig['L6']
# test_df_orig['Slice'] = test_df_orig['L6']

# train_df_orig['Slice'] = train_df_orig['L1'].astype(str) + "_" + train_df_orig['Group'].astype(str)
# test_df_orig['Slice'] = test_df_orig['L1'].astype(str) + "_" + test_df_orig['Group'].astype(str)

# train_df_orig['Slice'] = train_df_orig['Location'].astype(str) + "_" + train_df_orig['Group'].astype(str)
# test_df_orig['Slice'] = test_df_orig['Location'].astype(str) + "_" + test_df_orig['Group'].astype(str)

# iter_list = train_df_orig['Slice'].unique()
# iter_count = iter_list.size + 1

# for j in range(1, iter_count):
#     # j=1
try:
    # if env == "local":
    #     print(str(j) + " out of " + str(iter_count - 1) + " : " + iter_list[
    #     j - 1])
    # else:
    #     logger.debug(str(j) + " out of " + str(iter_count - 1) + " : " +
    #                  iter_list[
    #                      j - 1])

    # train_df = train_df_orig.copy()
    if env != "local":
        logger.debug("model run starting fwd")
    else:
        print("model run starting fwd")

    ###XGB RF##
    ##Select feature set##
    enc_cols = ['Item_Loc_te']
    feature_cols = ts_cols + enc_cols
    ## Creating the Training matrix with all feature columns##
    X = train_df.loc[:, feature_cols]
    ## Creating response vector
    y = train_df['Qty']
    # xgb_rf = XGBRFRegressor(random_state=42)
    if env != "local":
        logger.debug("xgb_rf model run starting")
    else:
        print("xgb_rf model run starting")
    xgb_rf = XGBRFRegressor(n_estimators=1000, random_state=42)
    # fit the model
    start_time = time.time()
    xgb_rf.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("xgb_rf --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("xgb_rf --- %s seconds ---" % (time.time() -
                                                    start_time))

    # Feature_Imp_all = get_feature_imp(xgb_rf,'xgb_rf')
    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(xgb_rf,'xgb_rf'),
    #                         how = 'left', on = ['Time','Feature'])

    ###xgb##
    if env != "local":
        logger.debug("xgb model run starting")
    else:
        print("xgb model run starting")
    xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01,
                       random_state=42)
    # fit the model
    start_time = time.time()
    xgb.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("xgb --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("xgb --- %s seconds ---" % (time.time() - start_time))

    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(xgb,'xgb'),
    #                   how = 'left', on = ['Week','Feature'])

    ###LGBM##
    ##Select feature set##
    feature_cols = feature_cols_orig
    ## Creating the Training matrix with all feature columns##
    train_df = train_df.dropna(subset=['Product Segment'])

    X = train_df.loc[:, feature_cols]
    ## Creating response vector
    y = train_df['Qty']
    if env != "local":
        logger.debug("lgb model run starting")
    else:
        print("lgb model run starting")
    lgb = LGBMRegressor(n_estimators=2000, learning_rate=0.005,
                        max_depth=10, num_leaves=int((2 ** 10) / 2),
                        max_bin=1000,
                        random_state=42)
    # lgb = LGBMRegressor(random_state=42, verbose = -1)
    # fit the model
    start_time = time.time()
    lgb.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("lgb --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("lgb --- %s seconds ---" % (time.time() - start_time))

    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(lgb,'lgb'),
    #                   how = 'left', on = ['Week','Feature'])

    ###Catboost##
    # cb = CatBoostRegressor(n_estimators=5000, learning_rate = 0.005,
    #                        # cat_features = cat_cols, one_hot_max_size = 16,
    #                        random_state=42, verbose=0)
    if env != "local":
        logger.debug("catb model run starting")
    else:
        print("catb model run starting")
    cb = CatBoostRegressor(n_estimators=1000, learning_rate=0.01,
                           cat_features=cat_cols, one_hot_max_size=16,
                           random_state=42, verbose=0)
    # fit the model
    start_time = time.time()
    cb.fit(X, y, sample_weight=train_df.weight)

    if env == "local":
        print("cb --- %s seconds ---" % (time.time() - start_time))
    else:
        logger.debug("cb --- %s seconds ---" % (time.time() - start_time))

    # Feature_Imp_all = pd.merge(Feature_Imp_all, get_feature_imp(cb,'cb'),
    #                   how = 'left', on = ['Time','Feature'])

    ########################Predicting########################
    # test_df = test_df_lr.copy()

    #####Prediction#####
    feature_cols = ts_cols + enc_cols
    y_pred = xgb_rf.predict(test_df_lr.loc[:, feature_cols])
    test_df_lr['xgb_rf'] = y_pred
    test_df_lr['xgb_rf'][test_df_lr['xgb_rf'] < 0] = 0

    y_pred = xgb.predict(test_df_lr.loc[:, feature_cols])
    test_df_lr['xgb'] = y_pred
    test_df_lr['xgb'][test_df_lr['xgb'] < 0] = 0

    feature_cols = feature_cols_orig
    y_pred = lgb.predict(test_df_lr.loc[:, feature_cols])
    test_df_lr['lgb'] = y_pred
    test_df_lr['lgb'][test_df_lr['lgb'] < 0] = 0

    y_pred = cb.predict(test_df_lr.loc[:, feature_cols])
    test_df_lr['cb'] = y_pred
    test_df_lr['cb'][test_df_lr['cb'] < 0] = 0

    # # Appending Feature Imp
    # if j == 1:
    #     Feature_Imp_df = Feature_Imp_all
    # else:
    #     Feature_Imp_df = Feature_Imp_df.append(Feature_Imp_all)

    # Appending test_df_lr
    # if j == 1:
    #     test_df_lr_all = test_df_lr
    # else:
    #     test_df_lr_all = test_df_lr_all.append(test_df_lr)

    # test_df_lr_all = test_df_lr.copy()

    if env == "local":
        print("test_df_lr")
        print(test_df_lr.head(10))
    else:
        logger.debug("test_df_lr")
        logger.debug(test_df_lr.head(10))
        logger.debug(test_df_lr.dtypes)
        logger.debug("test_df_lr")
        import datetime

        t = datetime.date(2022, 11, 6)
        logger.debug(test_df_lr[(test_df_lr['Week'] == t) & (
                    test_df_lr['Item'] == '000000447000198300') & (
                                            test_df_lr['Location'] == '0457')])

except:
    pass

####Outputting####
output_df = test_df_lr[['Sales Org', 'Location', 'Item', 'Week', 'Qty',
                        'xgb_rf', 'xgb', 'lgb', 'cb']]

####Merging lag 2 week with lr output####
output_df_l2.columns = ['Sales Org', 'Location', 'Item', 'Week', 'Qty',
                        'xgb_rf_l2', 'xgb_l2', 'lgb_l2', 'cb_l2']

output_df = pd.merge(output_df, output_df_l2, how='left',
                     on=['Sales Org', 'Location', 'Item', 'Week'])

output_df['xgb_rf'] = np.where(output_df['xgb_rf_l2'].notna(),
                               output_df['xgb_rf_l2'], output_df['xgb_rf'])
output_df['xgb'] = np.where(output_df['xgb_l2'].notna(), output_df['xgb_l2'],
                            output_df['xgb'])
output_df['lgb'] = np.where(output_df['lgb_l2'].notna(), output_df['lgb_l2'],
                            output_df['lgb'])
output_df['cb'] = np.where(output_df['cb_l2'].notna(), output_df['cb_l2'],
                           output_df['cb'])

if env != "local":
    logger.debug(
        "Shape of output df before final format and after merge with output df l2")
    logger.debug(output_df.shape)
    logger.debug(
        "output df before final format and after merge with output df l2")
    logger.debug(output_df.head())
    logger.debug(output_df.dtypes)
    import datetime

    t = datetime.date(2022, 11, 6)
    logger.debug(output_df[(output_df['Week'] == t) & (
                output_df['Item'] == '000000447000198300') & (
                                       output_df['Location'] == '0457')])

# ###Disaggregation Logic based on last 6 weeks###
# nw = 6
# ll_df = tran_df[(tran_df['Week'] > hist_week - np.timedelta64(nw, 'W')) & (
#             tran_df['Week'] <= hist_week)]
# ll_df = ll_df.groupby(['Sales Org', 'Location', 'Customer',
#                        'Item', 'EPM'])['Qty'].sum().reset_index()
# ll_df.rename(columns={'Qty': 'Qty_ll'}, inplace=True)
#
# hl_df = tran_df[(tran_df['Week'] > hist_week - np.timedelta64(nw, 'W')) & (
#             tran_df['Week'] <= hist_week)]
#
# hl_df = hl_df.groupby(['Sales Org', 'Location',
#                        'Item', ])['Qty'].sum().reset_index()
# hl_df.rename(columns={'Qty': 'Qty_hl'}, inplace=True)
#
# hl_df = pd.merge(hl_df, ll_df, how='left',
#                  on=['Sales Org', 'Location', 'Item'])
# hl_df['perc'] = hl_df['Qty_ll'] / hl_df['Qty_hl']
# hl_df['perc'] = hl_df['perc'].fillna(0)
#
# ###Disaagreagating the forecast###
# # temp = output_df
# output_df = pd.merge(output_df, hl_df, how='left',
#                      on=['Sales Org', 'Location', 'Item'])
# output_df['MLL1'] = output_df['xgb_rf'] * output_df['perc']
# output_df['MLL2'] = output_df['xgb'] * output_df['perc']
# output_df['MLL3'] = output_df['lgb'] * output_df['perc']
# output_df['MLL4'] = output_df['cb'] * output_df['perc']
# output_df['MLL5'] = output_df[['MLL1', 'MLL2', 'MLL3', 'MLL4']].mean(axis=1,
#                                                                      skipna=True)
#
# # temp['cb'].sum()
# # output_df['MLL5'].sum()
#
# # Appending output df
# if c == 1:
#     output_df_all = output_df
# else:
#     output_df_all = output_df_all.append(output_df)
#
# if env != "local":
#     logger.debug("Shape of output df after appending")
#     logger.debug(output_df.shape)

##########End backtest loop here###########

if env == "local":
    output_df.to_csv(
        cwd + 'output_df_' + reg + '_' + str(date.today()) + '.csv',
        index=False)

###Foramtting in o9 #####
o9_df = pd.merge(output_df, time_df, how='left', on='Week')
o9_df['Version'] = version

# o9_df = o9_df[['Version', 'Customer', 'Location', 'EPM',
#                'Item', 'WeekStr', 'MLL1', 'MLL2', 'MLL3', 'MLL4', 'MLL5']]

o9_df = o9_df[['Version', 'Sales Org', 'Location',
               'Item', 'WeekStr', 'xgb_rf', 'xgb', 'lgb', 'cb']]

o9_df.rename({'Version': 'Version.[Version Name]',
              'Sales Org': 'Sales Domain.[Sales Org]',
              'Location': 'Location.[Location]', 'Item': 'Item.[Planning Item]',
              'WeekStr': 'Time.[Week]', 'cb': 'MLL Model 1 Fcst HL',
              'lgb': 'MLL Model 2 Fcst HL', 'xgb_rf': 'MLL Model 3 Fcst HL',
              'xgb': 'MLL Model 4 Fcst HL'}, axis=1, inplace=True)

# o9_df.columns = ['Version.[Version Name]', 'Sales Domain.[Sales Org]',
#                  'Location.[Location]', 'Item.[Planning Item]',
#                  'Time.[Week]', 'MLL Model 1 Fcst HL Lag 2', 'MLL Model 2 Fcst HL Lag 2',
#                  'MLL Model 3 Fcst HL Lag 2', 'MLL Model 4 Fcst HL Lag 2']

# o9_df_orig = o9_df.copy()
# o9_df['Item.[Planning Item]'] = o9_df['Item.[Planning Item]'].str.zfill(18)
# o9_df['Location.[Location]'] = o9_df['Location.[Location]'].str.zfill(4)

if env != "local":
    logger.debug("Shape of o9 df")
    logger.debug(o9_df.shape)
    logger.debug("o9_df head")
    logger.debug(o9_df.head(10))
    logger.debug(o9_df[(o9_df['Time.[Week]'] == '2022-W46') & (
                o9_df['Item.[Planning Item]'] == '000000447000198300') & (
                                   o9_df['Location.[Location]'] == '0457')])

if env == "local":
    o9_df.to_csv(
        cwd + 'Fact.MLL_HL_45' + reg + '_' + str(date.today()) + '.csv',
        index=False)
else:
    O9DataLake.put("o9_df", o9_df)

###################################################
if env != "local":
    logger.debug("Script completed!")
